# Standardize survey data

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pandas import ExcelFile

SURVEY_DATA_COLUMNS=['species','latitude','longitude','phase','presence','magnitude']

## Input and output

In [2]:
DB='../output/survey.sqlite'
PHASE_ONE='../../data/field_survey/field_report_2019-06-11_phase1.xlsx'
PHASE_TWO='../../data/field_survey/field_report_2019-08-07_phase2.csv'

## Initiating dataframe
Mandatory column names for database (more can be added): `index,species,latitude,longitude,presence`

In [3]:
surveyData=pd.DataFrame(columns=SURVEY_DATA_COLUMNS)

The function `finalize()` below performs final cleaning up and appending to main dataframe.

In [4]:
def finalize(df,surveyData):
    surveyData=pd.concat([surveyData,df],sort=True)
    print("After appending to main dataframe: ",surveyData.shape)
    return surveyData

## Phase 1
Processing one sheet at a time. The below function reads in a sheet and modifies columns.
Then, we process each species individually. Note that there are a number of duplicate entries.

In [5]:
def process_phase_one_sheet(sheetName,speciesName):
    df=pd.read_excel(PHASE_ONE,sheet_name=sheetName)
    df.columns=map(str.lower,df.columns)
    print("Original data shape: ",df.shape)
    
    if speciesName=='Parthenium hysterophorus':
        df=df[['latitude','longitude','magnitude','presence_absence']]
        df['presence']=df['presence_absence']=='Presence'
        df=df.drop(['presence_absence'],axis=1)
    elif speciesName=='Eichhornia crassipes':
        df=df[['x_coordi','y_coordi','magnitude']]
        df.columns=['latitude','longitude','magnitude']
    else:
        df=df[['latitude','longitude','magnitude']]
    
    df=df.drop_duplicates()
    df=df[df.latitude!='Absent']
    df=df[df.longitude!='Absent']
    print("After dropping duplicates and extra columns: ",df.shape)
    if speciesName=='Parthenium hysterophorus':
        df.loc[df['magnitude'].isna(),'magnitude']='Not recorded.'
    else:
        df=df[~df['magnitude'].isna() & ~df['latitude'].isna() & ~df['longitude'].isna()]
    print("After dropping rows with magnitude=NaN: ",df.shape)
    df['presence']=(df['magnitude']!='Absent') | (df['magnitude']!='')
    df['species']=speciesName
    df['phase']=1

    return df

### Lantana camara

In [6]:
df=process_phase_one_sheet('Lentena','Lantana camara')
surveyData=finalize(df,surveyData)

Original data shape:  (538, 12)
After dropping duplicates and extra columns:  (514, 3)
After dropping rows with magnitude=NaN:  (514, 3)
After appending to main dataframe:  (514, 6)


//anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Chromolaena odorata

In [7]:
df=process_phase_one_sheet('Chromoleana','Chromolaena odorata')
surveyData=finalize(df,surveyData)

Original data shape:  (121, 12)
After dropping duplicates and extra columns:  (120, 3)
After dropping rows with magnitude=NaN:  (120, 3)
After appending to main dataframe:  (634, 6)


###  Ipomoea carnea

In [8]:
df=process_phase_one_sheet('Ipomea','Ipomoea carnea')
surveyData=finalize(df,surveyData)

Original data shape:  (288, 11)
After dropping duplicates and extra columns:  (287, 3)
After dropping rows with magnitude=NaN:  (286, 3)
After appending to main dataframe:  (920, 6)


### Mikania micrantha

In [9]:
df=process_phase_one_sheet('Mikeniea','Mikania micrantha')
surveyData=finalize(df,surveyData)

Original data shape:  (116, 12)
After dropping duplicates and extra columns:  (116, 3)
After dropping rows with magnitude=NaN:  (115, 3)
After appending to main dataframe:  (1035, 6)


### Ageratina adenophora

In [10]:
df=process_phase_one_sheet('Ageratina','Ageratina adenophora')
surveyData=finalize(df,surveyData)

Original data shape:  (543, 11)
After dropping duplicates and extra columns:  (521, 3)
After dropping rows with magnitude=NaN:  (520, 3)
After appending to main dataframe:  (1555, 6)


### Parthenium hysterophorus

In [11]:
df=process_phase_one_sheet('Parthenium','Parthenium hysterophorus')
surveyData=finalize(df,surveyData)

Original data shape:  (380, 12)
After dropping duplicates and extra columns:  (335, 4)
After dropping rows with magnitude=NaN:  (335, 4)
After appending to main dataframe:  (1890, 6)


### Eichhornia crassipes

In [12]:
df=process_phase_one_sheet('Eichorniea','Eichhornia crassipes')
surveyData=finalize(df,surveyData)

Original data shape:  (62, 10)
After dropping duplicates and extra columns:  (62, 3)
After dropping rows with magnitude=NaN:  (51, 3)
After appending to main dataframe:  (1941, 6)


## Phase 2
This is present in a single csv file.

In [13]:
#df=pd.read_csv(PHASE_TWO)
#df=df[['longitude','latitude','Species','Field  Validation status']]
#df.columns=['longitude','latitude','species','presence']
#df['presence']=df['presence']=='Present'
#df['phase']=2
#print("Original data shape",df.shape)
#df=df.dropna()
#print("After dropping rows with NaNs", df.shape)
#surveyData=finalize(df,surveyData)
#surveyData

## Push to database
Assumes that the database has the following table:
```
CREATE TABLE "survey" (
	"species"	TEXT,
	"latitude"	REAL,
	"longitude"	REAL,
    "phase"	INTEGER,
	"presence"	INTEGER,
	"magnitude"	TEXT,
	PRIMARY KEY("species","latitude","longitude","phase")
);
```

In [14]:
conn = sqlite3.connect(DB)
cur=conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS "survey" ( \
    "species"    TEXT, \
    "latitude"    REAL, \
    "longitude"    REAL, \
    "phase"    INTEGER, \
    "presence"    INTEGER, \
    "magnitude"    TEXT, \
    PRIMARY KEY("species","latitude","longitude","phase") \
);')
surveyData.to_sql('temporary_table',conn,if_exists='replace')
cur=conn.cursor()
cur.execute('INSERT OR IGNORE INTO survey SELECT ' + ','.join(SURVEY_DATA_COLUMNS) + ' FROM temporary_table;')
cur.execute('SELECT count(*) FROM survey;')
numRows=cur.fetchall()
cur.execute('DROP TABLE temporary_table;')
conn.commit()
conn.close()

print("Number of rows in database: %d" %numRows[0])
print("Number of rows in dataframe: %d" %surveyData.shape[0])
# surveyData[surveyData[['species','latitude','longitude','phase']].duplicated(keep=False)].sort_values(by=['longitude'])

Number of rows in database: 1886
Number of rows in dataframe: 1941


**Note:** The discrepancy in the number of rows in database and datagra